install ChromaDB & Gemini API Python Library.

In [ ]:
!pip install -U -q google.generativeai
!pip install -q chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.5/664.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies 

import the modules.

In [ ]:
import textwrap
import chromadb
import numpy as np
import pandas as pd

import google.generativeai as genai
import google.ai.generativelanguage as glm

# Used to securely store your API key 存取放在secreat 的 key
from google.colab import userdata

from IPython.display import Markdown #用來印出data的
from chromadb import Documents, EmbeddingFunction, Embeddings

get API Key from secret

In [ ]:
API_KEY=userdata.get('API_KEY')
genai.configure(api_key=API_KEY)

to know what kinds of related models we can use.

In [ ]:
for model in genai.list_models():
  if 'embedContent' in model.supported_generation_methods:
    print(model.name)

models/embedding-001
models/text-embedding-004


Put the data that will be embedded and put into the vector database. the original example provides three document about Google products, and here I replaced these three with three recent news article, which are about earthquake, Shohei Ohtani, and Vietnam tycoon. Trying to test if I can retrieve the relevent passage beased on the key words I provide from the vector database, which contains the vector infomation of the three documents, and also to test if Gemini can give me reasonable response base on the prompt I give that containing query question and related passage fetched from calling 'query' on the vector database to find similar embeddings by performing nearest neighbor search.  

In [ ]:
DOCUMENT1 = "A court in Vietnam sentenced real estate tycoon Truong My Lan Thursday to death over her role in a 304 trillion dong ($12.46 billion) financial fraud case, the country’s biggest on record, state media reported.Her trial, begun on March 5 and ending earlier than planned, was one dramatic result of a campaign against corruption that the leader of the ruling Communist Party, Nguyen Phu Trong, has pledged for years to stamp out. Lan, the chairwoman of real estate developer Van Thinh Phat Holdings Group, was found guilty of embezzlement, bribery and violations of banking rules at the end of a trial in the business hub of Ho Chi Minh City, state media said.“We will keep fighting to see what we can do,” a family member told Reuters, speaking on condition of anonymity. Before the verdict he had said Lan would appeal against the sentence.Lan had pleaded not-guilty to the embezzlement and bribery charges, Nguyen Huy Thiep, one of Lan’s lawyers told Reuters.“Of course she will appeal the verdict,” he added noting she was sentenced to death for the embezzlement charge and to 20 years each for the other two charges of bribery and violations of banking regulations.Vietnam imposes the death penalty mostly over violent offenses but also for economic crimes. Human rights groups say it has executed hundreds of convicts in recent years, mainly by lethal injection.The Thanh Nien newspaper said 84 defendants in the case received sentences ranging from probation for three years to life imprisonment. Among them are Lan’s husband, Eric Chu, a businessman from Hong Kong, who was sentenced to nine years in jail, and her niece who got 17 years."
DOCUMENT2 = "A former interpreter for Los Angeles Dodgers superstar Shohei Ohtani is in negotiations to plead guilty to federal crimes related to accusations he stole millions from Ohtani and used it for gambling, The New York Times reported Wednesday, citing three people familiar with the matter. Ippei Mizuhara was fired last month after Ohtani’s lawyers accused him of stealing from Ohtani and placing bets with a bookmaker who is under federal investigation. Ohtani later alleged Mizuhara stole the money from one of his bank accounts.Prosecutors now believe that Mizuhara could have stolen money from Ohtani beyond the $4.5 million that Mizuhara initially was accused of taking, the three people with knowledge of the matter told the New York Times.Law enforcement investigators believe they have evidence that Mizuhara was able to change the settings on Ohtani’s bank accounts, which would have prevented Ohtani from getting alerts and confirmation on the transactions, the New York Times reported, citing the three people knowledgeable of the case.Ohtani has denied any involvement in the scandal.An attorney for Mizuhara, Michael Freedman, told CNN Wednesday that“we do not have any comment at this time.”The allegations became public while the Dodgers were in South Korea for an MLB season-opening series in March, with ESPN and the Los Angeles Times reporting Ohtani’s lawyers accused Mizuhara of “massive theft” of millions of dollars and placing bets with a bookie under federal investigation. Mizuhara was fired while the team was in Seoul. Ohtani, who during the offseason signed baseball’s richest contract with the Dodgers, adamantly denied to reporters March 25 betting on any sport or paying a bookie. Also, elaborating on his lawyers’ accusations that Mizuhara stole from him, he alleged Mizuhara took money from one of his bank accounts.“I never bet on baseball or any other sports or never have asked somebody to do it on my behalf and I have never gone through a bookmaker to bet on sports,” Ohtani said through a different interpreter during the March 25 news conference at Dodger Stadium.The two-time MVP said he didn’t know money was being sent from one of his bank accounts to a bookmaker until he was on the South Korea trip, when he met one-on-one with after a team meeting in the clubhouse that left the slugger-pitcher thinking something was wrong.“I’m very saddened and shocked that someone who I’ve trusted has done this,” Ohtani, Major League Baseball’s only two-way player, said. ESPN’s Tisha Thompson, citing multiple unnamed sources, said on CNN’s “The Lead” last month at least $4.5 million was withdrawn via wire transfer from Ohtani’s bank accounts, though it was unclear who initiated the transfers. The scandal brought renewed scrutiny to professional sports’ closeness with gambling.The Internal Revenue Service and Major League Baseball have said they are investigating the matter, with the league trying to determine whether its strict rules against gambling were broken. A player or employee who gambles on baseball can be suspended for a year, while someone who gambles on a game they are in some way a part of can be suspended for life. MLB’s commissioner can also penalize someone who goes outside legal sports betting entities and gambles with a bookie.MLB considers Rule 21 so important it requires it be posted in every clubhouse.Mizuhara told ESPN in March:“I never bet on baseball … That’s 100 percent. I knew that rule. … We have a meeting about that in spring training.”Ohtani signed an unparalleled 10-year, $700 million deal with the Dodgers this offseason after winning the American League MVP award twice over the past three seasons as a pitcher and designated hitter with the Angels."

DOCUMENT3 = "The deadly 7.4 magnitude earthquake that hit Taiwan on Wednesday and left at least 9 people dead, also damaged 770 buildings, according to the latest estimates from the island’s National Fire Agency (NFA).In the capital Taipei, just 80 miles away from the epicenter, buildings also shook violently during what was Taiwan’s strongest earthquake in 25 years. But in a triumph of modern engineering, the soaring Taipei 101, once the world’s tallest skyscraper, emerged from the island’s latest seismic event undamaged.Footage from the quake appears to show the 1,667-foot-tall tower slightly swaying, its structural flexibility helping to counter the powerful motion of the quake. This movement perfectly demonstrated how the skyscraper’s single most important defense against earthquakes is the very material it is constructed from: reinforced concrete.Combining concrete’s compressive strength with steel’s tensile strength, the material makes the building flexible enough to sway, yet rigid enough to resist high winds and the typhoons that frequently hit Taiwan. (The principle that buildings can endure seismic forces by moving with them, rather than against them, has underpinned traditional architecture in earthquake-prone East Asian countries for centuries, from Japanese pagodas to Chinese palaces.)High within the tower, however, another technological innovation helps protect the 101-story skyscraper — a huge orb-like device known as a tuned mass damper. Suspended from 92 thick cables between the 87th and 92nd floors, the golden steel sphere can move about 5 feet in any direction. As a result, it acts like a pendulum that counteracts (or “dampens”) swaying motions.“It’s essentially a very large counterweight,” explained Stefan Al, author of “Supertall: How the World’s Tallest Buildings Are Reshaping Our Cities and Our Lives,” in a phone interview. “In the case of Taipei 101, it’s 660 tons. That seems really heavy, but if you compare it to the overall weight of the building, it’s just a fraction.“When a building starts to shake, (the tuned mass damper) will move in the opposite direction. In the case of Taipei 101, it's suspended… so it will lag behind as the tower sways and will absorb the kinetic energy by moving in the opposite direction,” Al said, explaining that hydraulic cylinders between the ball and the building convert this energy into heat, which is then dispersed.Tuned mass dampers are used in skyscrapers around the world, including the “super-skinny” Steinway Tower in New York and Dubai’s sail-shaped Burj al-Arab, which has 11 of them. The device crucially protects against the violent motion caused by “harmonic vibration,” which can cause structural failure during an earthquake, Al said.“(That’s) when buildings start to vibrate at their own resonance,” he said, comparing the phenomenon to a tuning fork. “This can lead to a collapse, (as the building) will start shaking faster and faster and faster.”Tuned mass dampers, which are also known as harmonic absorbers, are “tuned” to resonate at the same frequency as the building — but with wavelengths starting earlier or later, helping to dissipate the potentially catastrophic energy.The stability they provide can also reduce the uncomfortable — or even nauseating — effect swaying can have on a building’s occupants in high winds.Designed by Taiwanese firm C.Y. Lee & Partners, Taipei 101 was the world’s tallest building from 2004 until 2007, when it was surpassed by the Burj Khalifa in Dubai. An observation deck overlooking its groundbreaking tuned mass damper is now a popular visitor attraction, especially when it moves during strong winds.The giant sphere is not, however, the only design feature helping stabilize the tower, which stands close to a major fault line.For one, the skyscraper sits on exceptionally deep foundations — namely 380 reinforced concrete and steel piles drilled down into the bedrock below. Above them, the building’s core is connected to a series of “mega-columns,” located around its perimeter, via enormous steel outrigger trusses.The high-rise complies with strict anti-seismic building codes that are as stringent as one might expect on an island located along the “Pacific Ring of Fire,” which runs around the edge of the Pacific Ocean and causes massive seismic and volcanic activity from Indonesia to Chile.But while the design also underwent extensive digital modeling and “shake table” testing (in which scale models are tested on a device that replicates the movement of an earthquake), how a building like Taipei 101 would react to a stronger or closer seismic event remains, to an extent, theoretical.“Even though we have computer simulations, there’s still something about the physical that that we cannot really get from digital simulations,” said Al, adding: “Despite our technological advances, we are still testing (designs) in wind tunnels and shaker tables."

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

creating the embedding database with ChromaDB and model embedding-001.

Task Type includes:

*   RETRIEVAL_QUERY
*   RETRIEVAL_DOCUMENT
*   SEMANTIC_SIMILARITY
*   CLASSIFICATION
*   CLUSTERING


In [ ]:
class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    model = 'models/embedding-001'
    title = "Custom query"
    return genai.embed_content(model=model,
         content=input,task_type="retrieval_document",title=title)["embedding"]
def create_chroma_db(documents, name):
  chroma_client = chromadb.Client()
  db = chroma_client.create_collection(name=name, embedding_function=GeminiEmbeddingFunction())

  for i, d in enumerate(documents):
    db.add(
      documents=d,
      ids=str(i)
    )
  return db
#using create database function to do document embedding and put the result in to the database
db = create_chroma_db(documents, "newssearch")
#see the result
pd.DataFrame(db.peek(3))


,ids,embeddings,metadatas,documents,uris,data
0,0,"[0.013291151262819767, 0.01081881858408451, -0...",None,A court in Vietnam sentenced real estate tycoo...,None,None
1,1,"[-0.015667986124753952, -0.01856137625873089, ...",None,A former interpreter for Los Angeles Dodgers s...,None,None
2,2,"[0.016905754804611206, -0.035177651792764664, ...",None,The deadly 7.4 magnitude earthquake that hit T...,None,None


getting the relevant document and use the result to make prompt.

In [ ]:
def get_relevant_passage(query, db):
  passage = db.query(query_texts=[query], n_results=1)['documents'][0][0]
  return passage

def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = ("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a audience who want to get infomation in conscise way, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

# Perform embedding search
query = "who is the interpreter of ohtani and what the interpreter was accused? "
passage = get_relevant_passage(query, db)
Markdown(passage)
#pass a query to prompt
prompt = make_prompt(query, passage)
Markdown(prompt)

#use the generate_content method to generate a reponse from the model base on the prompt
model = genai.GenerativeModel('gemini-pro')
answer = model.generate_content(prompt)
Markdown(answer.text)

Ippei Mizuhara was the interpreter for Los Angeles Dodgers superstar Shohei Ohtani. He was accused of stealing millions from Ohtani, which he allegedly used for gambling.